<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=50bc597ab2f8ff1a5f8e0b8d3f7379fd24f32b8a8b180fd9a7bcd78fc508b0e8
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:


      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-06 09:32:45
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.44 C
-------------------
Today PnL: -44.87 K (-0.31%)
Current PnL: -20.50 L (-13.43%)
CY Booked + Current PnL: -7.85 L (-5.14%)
-------------------
Total profit:  2.11 L
Total loss:  -22.61 L
-------------------
Total Booked + Current PnL: 19.13 L (15.23%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.78%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.48 L (60.72%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 8.04%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,66.0,X-LC,5.80,232518.0,22152.0,6627.0,1.18,10.53,2.85,13.68,37.0,3.34,1.63,28.79,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-3.88,68.0,M-LC,10.70,172881.0,32041.0,12447.0,0.34,22.75,7.20,31.59,66.0,2.57,1.21,55.26,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,101.92,52.0,M-SC,1.75,86971.0,-13806.0,13898.0,-0.73,-13.70,15.98,0.09,245.0,-0.99,0.61,14.26,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.45,41.0,X-MC,2.60,88380.0,-2298.0,17208.0,0.34,-2.53,19.47,16.44,101.0,-0.13,0.62,17.13,X40,ATH,PHARMA
33,ICICIGI,2252.93,-13.14,59.0,X-MC,5.57,149251.0,13118.0,17462.0,-1.13,9.64,11.70,22.47,91.0,0.75,1.04,23.87,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.40,-13.20,65.0,M-SC,27.15,83626.0,-93189.0,173984.0,8.07,-52.70,208.05,45.70,197.0,-0.54,0.58,9.17,XY24,NTT,TRAVEL
6,ASIANPAINT,3465.66,-9.66,68.0,X-LC,4.62,227911.0,-23857.0,77057.0,4.15,-9.48,33.81,21.13,27.0,-0.31,1.59,22.59,X40,ATH,PAINTS
67,SIS,528.00,2050.78,51.0,H-SC,2.69,86789.0,-24243.0,47847.0,2.82,-21.83,55.13,21.26,156.0,-0.51,0.61,17.26,OX40N,NTT,MISC
55,RAJOOENG,143.10,-42.01,48.0,H-SC,17.12,87880.0,-14620.0,55224.0,2.08,-14.26,62.84,39.61,114.0,-0.26,0.61,7.33,AR,ATH,MISC
13,BERGEPAINT,680.00,-17.39,58.0,X-MC,1.30,222163.0,-5176.0,53230.0,2.07,-2.28,23.96,21.14,106.0,-0.10,1.55,25.22,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.00,31.80,46.0,M-SC,4.37,173969.0,-88249.0,158068.0,-5.99,-33.65,90.86,26.63,229.0,-0.56,1.22,8.16,XY24,NTT,MISC
85,WHIRLPOOL,2270.00,-40.66,52.0,M-SC,5.30,123041.0,8243.0,85796.0,-2.69,7.18,69.73,81.92,223.0,0.10,0.86,45.66,XR,NTT,DURABLES
32,HONAUT,58357.33,-23.86,40.0,X-SC,1.12,107310.0,-20628.0,67766.0,-2.65,-16.12,63.15,36.84,143.0,-0.30,0.75,9.86,X40N,ATH,ELECTRICAL
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.03,4.00,XY24,NTT,AUTO
59,REPCOHOME,880.00,-52.31,70.0,H-SC,7.36,272847.0,-13658.0,270992.0,-1.93,-4.77,99.32,89.82,134.0,-0.05,1.91,44.36,XY24,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,49.0,M-LC,4.96,204242.0,1072.0,41053.0,0.19,0.53,20.10,20.74,55.0,0.03,1.43,7.43,XY25,NTT,FINANCE
31,HINDZINC,730.22,25.32,45.0,M-LC,10.47,204776.0,-300.0,112873.0,-0.55,-0.15,55.12,54.89,52.0,-0.00,1.43,23.91,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.92,52.0,M-SC,1.75,86971.0,-13806.0,13898.0,-0.73,-13.70,15.98,0.09,245.0,-0.99,0.61,14.26,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,44.0,H-SC,2.68,221481.0,-48186.0,84517.0,-0.99,-17.87,38.16,13.47,138.0,-0.57,1.55,12.62,XY24,NTT,PAINTS
17,CERA,9475.0,-21.72,54.0,H-SC,1.76,142853.0,-33050.0,75069.0,-1.33,-18.79,52.55,23.89,149.0,-0.44,1.00,23.37,OX40N,NTT,CERAMICS
67,SIS,528.0,2050.78,51.0,H-SC,2.69,86789.0,-24243.0,47847.0,2.82,-21.83,55.13,21.26,156.0,-0.51,0.61,17.26,OX40N,NTT,MISC
42,JCHAC,2282.0,19943.48,63.0,M-SC,1.52,97950.0,-29855.0,29845.0,0.09,-23.36,30.47,-0.01,232.0,-1.00,0.68,10.91,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-3.88,68.0,M-LC,10.7,172881.0,32041.0,12447.0,0.34,22.75,7.2,31.59,66.0,2.57,1.21,55.26,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,51.56,62.0,M-MC,10.01,239461.0,14499.0,153159.0,0.64,6.45,63.96,74.53,192.0,0.09,1.67,40.78,XY24,BTT,STEEL
56,RECLTD,446.00,45.36,49.0,M-LC,4.96,204242.0,1072.0,41053.0,0.19,0.53,20.10,20.74,55.0,0.03,1.43,7.43,XY25,NTT,FINANCE
46,KPIGREEN,731.05,10.56,63.0,H-SC,7.49,131620.0,6323.0,52609.0,-1.83,5.05,39.97,47.03,141.0,0.12,0.92,64.80,MH,ATH,POWER
79,UNIONBANK,163.00,-3.88,68.0,M-LC,10.70,172881.0,32041.0,12447.0,0.34,22.75,7.20,31.59,66.0,2.57,1.21,55.26,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.66,52.0,M-SC,5.30,123041.0,8243.0,85796.0,-2.69,7.18,69.73,81.92,223.0,0.10,0.86,45.66,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,10.56,63.0,H-SC,7.49,131620.0,6323.0,52609.0,-1.83,5.05,39.97,47.03,141.0,0.12,0.92,64.80,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-40.66,52.0,M-SC,5.30,123041.0,8243.0,85796.0,-2.69,7.18,69.73,81.92,223.0,0.10,0.86,45.66,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,5.86,130645.0,7309.0,124322.0,-0.13,5.93,95.16,106.72,119.0,0.06,0.91,30.69,AR,ATH,MISC
31,HINDZINC,730.22,25.32,45.0,M-LC,10.47,204776.0,-300.0,112873.0,-0.55,-0.15,55.12,54.89,52.0,-0.00,1.43,23.91,X5K,ATH,METALS
86,WIPRO,420.00,-15.61,42.0,M-LC,5.49,148298.0,-2647.0,112098.0,0.53,-1.75,75.59,72.51,53.0,-0.02,1.04,4.16,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.03,4.00,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.15,26.0,X-SC,6.91,87922.0,-40748.0,86049.0,-0.64,-31.67,97.87,35.20,219.0,-0.47,0.61,0.91,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-23.14,29.0,X-MC,4.70,192712.0,-7635.0,38138.0,-1.45,-3.81,19.79,15.23,178.0,-0.20,1.35,27.21,X40N,NTT,REALTY
51,PAGEIND,51769.93,-29.36,34.0,X-MC,7.23,79800.0,-3710.0,23740.0,-0.59,-4.44,29.75,23.98,82.0,-0.16,0.56,1.12,X40,ATH,APPARELS
53,QUESS,986.00,-49.86,35.0,X-SC,37.71,49851.0,-15155.0,166084.0,-0.97,-23.31,333.16,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-30.81,37.0,X-MC,0.93,200175.0,-645.0,68440.0,-0.42,-0.32,34.19,33.76,80.0,-0.01,1.40,4.86,X40,ATH,FMCG
41,ITC,452.00,-37.92,50.0,X-LC,1.11,201126.0,988.0,19911.0,0.59,0.49,9.90,10.44,4.0,0.05,1.41,6.08,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.25,46.0,H-LC,1.26,155500.0,-30595.0,78077.0,-0.31,-16.44,50.21,25.51,15.0,-0.39,1.09,14.59,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-7.60,39.0,X-MC,3.02,308175.0,-19816.0,126352.0,-1.27,-6.04,41.00,32.48,92.0,-0.16,2.15,4.69,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-4.95,73.0,X-LC,4.08,249004.0,14860.0,33765.0,-0.22,6.35,13.56,20.77,86.0,0.44,1.74,13.70,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.86,35.0,X-SC,37.71,49851.0,-15155.0,166084.0,-0.97,-23.31,333.16,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.15,26.0,X-SC,6.91,87922.0,-40748.0,86049.0,-0.64,-31.67,97.87,35.20,219.0,-0.47,0.61,0.91,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-29.36,34.0,X-MC,7.23,79800.0,-3710.0,23740.0,-0.59,-4.44,29.75,23.98,82.0,-0.16,0.56,1.12,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.74,41.0,X-MC,7.34,217600.0,-45765.0,155084.0,-0.68,-17.38,71.27,41.51,84.0,-0.30,1.52,2.11,XY25,ATH,FMCG
3,ACC,3906.00,-38.07,46.0,X-MC,2.44,183970.0,-53981.0,206635.0,0.39,-22.69,112.32,64.15,174.0,-0.26,1.29,3.20,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.86,35.0,X-SC,37.71,49851.0,-15155.0,166084.0,-0.97,-23.31,333.16,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.36,34.0,X-MC,7.23,79800.0,-3710.0,23740.0,-0.59,-4.44,29.75,23.98,82.0,-0.16,0.56,1.12,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.65,40.0,X-SC,4.33,80849.0,-64311.0,140241.0,-0.73,-44.30,173.46,52.31,136.0,-0.46,0.57,8.22,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.15,26.0,X-SC,6.91,87922.0,-40748.0,86049.0,-0.64,-31.67,97.87,35.20,219.0,-0.47,0.61,0.91,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,41.0,X-MC,2.60,88380.0,-2298.0,17208.0,0.34,-2.53,19.47,16.44,101.0,-0.13,0.62,17.13,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.01,48.0,X-LC,9.85,282818.0,-63138.0,129842.0,0.62,-18.25,45.91,19.28,1.0,-0.49,1.98,4.55,X40,ATH,IT
39,INFY,2275.00,-19.24,51.0,X-LC,5.12,314771.0,1883.0,169819.0,0.67,0.60,53.95,54.87,3.0,0.01,2.20,8.93,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,X-LC,1.11,201126.0,988.0,19911.0,0.59,0.49,9.90,10.44,4.0,0.05,1.41,6.08,X40,NTT,FMCG
83,VBL,671.64,-15.98,55.0,X-LC,6.85,302795.0,-13047.0,125720.0,0.76,-4.13,41.52,35.67,5.0,-0.10,2.12,9.15,X40N,ATH,FMCG
1,ABB,7934.00,-39.87,52.0,H-LC,5.08,251472.0,-10147.0,129357.0,-0.10,-3.88,51.44,45.57,7.0,-0.08,1.76,7.00,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,38.0,L-SC,27.79,76416.0,-37133.0,77188.0,-1.66,-32.70,101.01,35.27,268.0,-0.48,0.53,87.03,XR,NTT,HOTELS
7,ASIANTILES,137.00,7438.89,67.0,L-SC,10.63,82726.0,-11084.0,87706.0,-0.97,-11.82,106.02,81.67,269.0,-0.13,0.58,60.20,XR,NTT,CERAMICS
49,MASFIN,398.61,-16.89,57.0,H-SC,8.43,94680.0,-3300.0,24901.0,0.90,-3.37,26.30,22.05,152.0,-0.13,0.66,37.62,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.66,52.0,M-SC,5.30,123041.0,8243.0,85796.0,-2.69,7.18,69.73,81.92,223.0,0.10,0.86,45.66,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,5.86,130645.0,7309.0,124322.0,-0.13,5.93,95.16,106.72,119.0,0.06,0.91,30.69,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-52.31,70.0,H-SC,7.36,272847.0,-13658.0,270992.0,-1.93,-4.77,99.32,89.82,134.0,-0.05,1.91,44.36,XY24,NTT,FINANCE
81,VAIBHAVGBL,521.00,75.76,66.0,H-SC,2.33,152393.0,-30382.0,141451.0,-1.58,-16.62,92.82,60.77,125.0,-0.21,1.07,36.49,XR,NTT,JEWELLERY
46,KPIGREEN,731.05,10.56,63.0,H-SC,7.49,131620.0,6323.0,52609.0,-1.83,5.05,39.97,47.03,141.0,0.12,0.92,64.80,MH,ATH,POWER
62,SAMMAANCAP,326.00,-197.73,71.0,M-SC,33.10,168291.0,19071.0,125108.0,0.41,12.78,74.34,96.62,208.0,0.15,1.18,82.13,XY25,NTT,FINANCE
68,SONACOMS,806.63,-31.93,67.0,M-SC,6.91,84648.0,-16511.0,56511.0,-0.02,-16.32,66.76,39.54,202.0,-0.29,0.59,19.36,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.72
1,25,44.30
2,50,75.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.85
LC    33.14
MC    23.03
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.02
X40      16.18
X40N     12.44
XY25     11.94
XR        9.89
AR        7.84
OX40N     6.54
X200      1.78
X5K       1.43
SR        1.04
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.33
X-LC    22.43
X-MC    18.70
M-SC    12.60
X-SC     5.48
M-LC     5.11
H-LC     4.63
H-MC     2.34
M-MC     1.67
L-SC     1.44
L-LC     0.97
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.28,-3.60,37.09
IT,12.42,-18.48,81.95
FINANCE,12.22,-8.26,57.34
MISC,7.68,-18.19,77.81
ELECTRICAL,5.75,-9.87,48.82
PAINTS,5.55,-16.25,33.58
BANKS,4.07,-16.41,84.13
INSURANCE,3.83,-0.72,38.27
AUTO,2.74,-46.44,107.56


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3281831.0,22
XR,1244107.0,13
AR,1166014.0,9
X40,840230.0,12
X40N,691316.0,9
OX40N,550396.0,9
XY25,465963.0,8
SR,272967.0,2
X5K,112873.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3353417.0,24
M-SC,1243348.0,15
X-MC,1199468.0,14
X-LC,1070627.0,13
X-SC,566102.0,6
M-LC,278471.0,4
H-LC,277334.0,3
L-SC,254487.0,3
H-MC,254142.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1175811.0      6
           AR         834854.0      5
           XR         775321.0      7
M-SC       XY24       714387.0      6
X-MC       XY24       567148.0      4
X-LC       X40        458916.0      5
X-SC       X40N       313969.0      4
X-LC       XY24       311878.0      2
X-MC       X40        295265.0      6
H-SC       SR         272967.0      2
           OX40N      241855.0      3
X-LC       X40N       220603.0      3
H-LC       AR         207434.0      2
H-MC       XY24       180917.0      1
X-MC       XY25       180311.0      2
X-SC       XY24       166084.0      1
L-SC       XR         164894.0      2
X-MC       X40N       156744.0      2
M-MC       XY24       153159.0      1
M-SC       OX40N      145723.0      4
           XR         134404.0      2
           XY25       125108.0      1
           AR         123726.0      2
M-LC       X5K        112873.0      1
           XR         112098.0      1
L-SC       OX40N       89593.0      1
X-SC       X40         86049.0      1
X-LC       XY25        79230.0      3
H-MC       OX40N       73225.0      1
H-LC       X200        69900.0      1
L-MC       XR          57390.0      1
H-SC       MH          52609.0      1
M-LC       XY25        41053.0      1
L-LC       XY25        40261.0      1
M-LC       XY24        12447.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
